In [ ]:
import pandas as pd
import math
import numpy as np
from sklearn.neighbors import KernelDensity
import matplotlib.pyplot as plt
import copy
import statistics

data = pd.read_csv('Women.csv')
teams = ['USA', 'GBR', 'CAN', 'CHN', 'BRA', 'ITA', 'NED', 'FRA', 'JPN', 'AUS', 'ROU', 'KOR']


def points_before_olympic_games(modalidad_final, modalidad_qualifier): 
    if modalidad_final == 'Team Score Final':
        return team_points()
    dict_ = {}
    for i, _ in enumerate(data['Name']):

        # if pd.isna(data.iloc[i, data.columns.get_loc(modalidad_final)]):
        #     if pd.isna(data.iloc[i, data.columns.get_loc(modalidad_qualifier)]): continue
        #     else:
        #         if data['Name'][i] in dict_:
        #             dict_[data['Name'][i]].append(float(data[modalidad_qualifier][i]))
        #             continue
        #         else:
        #             dict_[data['Name'][i]] = [float(data[modalidad_qualifier][i])]
        #             continue

        if data[modalidad_final][i] == ' ':
            if data[modalidad_qualifier][i] == ' ': continue
            else:
                if data['Name'][i] in dict_:
                    dict_[data['Name'][i]].append(float(data[modalidad_qualifier][i]))
                    continue
                else:
                    dict_[data['Name'][i]] = [float(data[modalidad_qualifier][i])]
                    continue
        


        if math.isnan(float(data[modalidad_final][i])):
            if math.isnan(float(data[modalidad_qualifier][i])):
                continue
            else:
                if data['Name'][i] in dict_:
                    dict_[data['Name'][i]].append(float(data[modalidad_qualifier][i]))
                    continue
                else:
                    dict_[data['Name'][i]] = [float(data[modalidad_qualifier][i])]
                    continue
            
        if data['Name'][i] in dict_:
            dict_[data['Name'][i]].append(float(data[modalidad_final][i]))
        else:
            dict_[data['Name'][i]] = [float(data[modalidad_final][i])]
        

    return dict_

def team_points():
    tuple_arr = []
    dict_ = {}
    for i, _ in enumerate(data['Country']):
        if data['Team Score Final'][i] == ' ':
            if data['Team Score Qualif'][i] == ' ': continue
            else:
                if data['Name'][i] in dict_:
                    dict_[data['Name'][i]].append(float(data['Team Score Qualif'][i]))
                    continue
                else:
                    dict_[data['Name'][i]] = [float(data['Team Score Qualif'][i])]
                    continue

        if not data['Country'][i] in teams: continue
        if (data['Country'][i], data['Competition'][i]) in tuple_arr: continue
        else:
            if math.isnan(float(data['Team Score Final'][i])):
                if math.isnan(float(data['Team Score Qualif'][i])):
                    continue
                else:
                    tuple_arr.append((data['Country'][i], data['Competition'][i]))
                    if data['Country'][i] in dict_:
                        dict_[data['Country'][i]].append(float(data['Team Score Qualif'][i]))
                        continue
                    else:
                        dict_[data['Country'][i]] = [float(data['Team Score Qualif'][i])]
                        continue

            tuple_arr.append((data['Country'][i], data['Competition'][i]))
            if data['Country'][i] in dict_:
                dict_[data['Country'][i]].append(float(data['Team Score Final'][i]))
            else:
                dict_[data['Country'][i]] = [float(data['Team Score Final'][i])]
    return dict_

# def Simulate_Event(kernel_, dict_):
#     distributions = {}
#     for name in dict_:
#         c = np.array(dict_[name])
#         kde = KernelDensity(kernel=kernel_, bandwidth=0.75).fit(c.reshape(-1, 1))
#         distributions[name] = kde

#     dist_arr = []
#     for name in distributions:
#         dist_arr.append(distributions[name])
    
#     return distributions, dist_arr

def n_simulations(kernel_, dict_, n):
    predictions = {} # 'name': [points_simul_1, points_simul_2, ..., points_simul_n]
    for name, _ in dict_.items():
        predictions[name] = []
    

    for _ in range(n):
        for name, _ in dict_.items():
            
            c = np.array(dict_[name])
            try:
                kde = KernelDensity(kernel=kernel_, bandwidth=0.75).fit(X = c.reshape(-1, 1))
            except:
                predictions[name].append('foo')
                continue
            predictions[name].append(int(kde.sample(1)[0][0]))

    predictions1 = {}
    
    for name, _ in predictions.items():
        if 'foo' in predictions[name]:
            continue
        predictions1[name] = predictions[name]
    return predictions1


def Get_Rankings(dict_):
    new_dict = {}
    names_copy = []
    points_copy = []
    for name, _ in dict_.items():
        names_copy.append(name)
        points_copy.append(dict_[name])

    for i in range(len(dict_) - 1):
        for j in range(i, len(dict_)):
            if points_copy[i] < points_copy[j]:
                a = points_copy[i]
                b = names_copy[i]
                points_copy[i] = points_copy[j]
                names_copy[i] = names_copy[j]
                points_copy[j] = a
                names_copy[j] = b
    
    for i in range(len(names_copy)):
        new_dict[names_copy[i]] = i + 1
    
    return new_dict

#dict_ = points_before_olympic_games('All-Around Score Final', 'All-Around Score Qualif')
dict_ = points_before_olympic_games('Vault Score Final', 'Vault Score Qualif')
#dict_ = points_before_olympic_games('Uneven Bars Score Final', 'Uneven Bars Score Qualif')
#dict_ = points_before_olympic_games('Balance Beam Score Final', 'Balance Beam Score Qualif')
#dict_ = points_before_olympic_games('Floor Exercise Score Final', 'Floor Exercise Score Qualif')
#dict_ = points_before_olympic_games('Team Score Final', 'Team Score Qualif')
print(dict_)
n = 1000
predictions = n_simulations("tophat", dict_, n)
mean = {}
mode = {}
for name in predictions:
    mean[name] = statistics.mean(predictions[name])
    mode[name] = statistics.mode(predictions[name])

print(mean)
print(mode)

results_mean = Get_Rankings(mean)
results_mode = Get_Rankings(mode)

print(results_mean)
print(results_mode)


print('Resultados usando la media: \n')
i=1
for name, _ in results_mean.items():
    print(f'{i}: {name}\n ')
    i+=1
    if i==9: break

print('Resultados usando la moda: \n')
i=1
for name, _ in results_mode.items():
    print(f'{i}: {name}\n ')
    i+=1
    if i==9: break